## ResNet Original Training & Inference

#### Training ResNet50 with Cifar 10

In [3]:
from ResNet.resnet import *
from ResNet.train import *

resnet_train_cifar10(device="cuda")

Files already downloaded and verified
Files already downloaded and verified


TypeError: Value after * must be an iterable, not int